# Model

1. Dividir el set de datos entre valores dependientes (y) e independientes (X). Y sería la columna
que se quiere predecir (“churn”).
2. Generar sets de testeo y entrenamiento.
3. ¿Es necesario escalar las features? Hacerlo si fuera necesario.
4. Probar por lo menos dos modelos y seleccionar uno. Explicar porque lo selecciono y qué
métricas uso para decidir.

<hr/>